# bert-large-cased-whole-word-masking-finetuned-squad μοντέλο ,manual tokenization με το ίδιο μοντέλο και padding μέχρι μήκος 20 για κάθε πρόταση. Threshold μέσω μέσου όρου των συνημιτόνων των γνωστών hallucinations (p(Hallucination) = 1). Προκύπτουν 26 πιθανά hallucinations εκ των οποίων το 31.25% είναι σωστό.

Θεωρώντας hallucinations τα datapoint με p(Hallucination) >= 0.66 προκύπτουν 22 πιθανά hallucinations απο τα οποία το 28.57% είναι σωστά

Για το αρχείο processed_df.json έχουμε 42.85% accuracy με 31 predicted hallucinations (για p(Hallucination) >= 0.66)


Για το αρχείο processed_df.json έχουμε 40.625% accuracy με 31 predicted hallucinations (για p(Hallucination) = 1)


In [ ]:
pip install -U sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9f5e39e8752f35f053a4460f42cf2b4a3f8780229859aa166e83cabac323b4f1
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from transformers import BertModel, BertTokenizer
import torch
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

In [ ]:
model = BertModel.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad')

In [ ]:
with open("mt_processed_df.json", "r") as f:
    trial_data = json.load(f)

df = pd.json_normalize(trial_data)

#df.head(79)

In [ ]:
hallucinations = []
hallucinations_cosines = []
hallucinations_pred = []
cosine_similarities = []
output_pairs = []

for i in range(0, len(df)):

  sentence = df.iloc[i]['hyp']
  sentence_2 = df.iloc[i]['tgt']

  #tokens = df.iloc[i]['hyp_lemmatized']
  #tokens_2 = df.iloc[i]['tgt_lemmatized']

  tokens = tokenizer.tokenize(sentence)
  tokens_2 = tokenizer.tokenize(sentence_2)

  tokens = ['[CLS]'] + tokens + ['[SEP]']
  tokens_2 = ['[CLS]'] + tokens_2 + ['[SEP]']

  while len(tokens) < 20:
    tokens.append('[PAD]')
  while len(tokens_2) < 20:
    tokens_2.append('[PAD]')

  attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]
  attention_mask_2 = [1 if i!= '[PAD]' else 0 for i in tokens_2]

  token_ids = tokenizer.convert_tokens_to_ids(tokens)
  token_ids_2 = tokenizer.convert_tokens_to_ids(tokens_2)

  token_ids = torch.tensor(token_ids).unsqueeze(0)
  token_ids_2 = torch.tensor(token_ids_2).unsqueeze(0)

  attention_mask = torch.tensor(attention_mask).unsqueeze(0)
  attention_mask_2 = torch.tensor(attention_mask_2).unsqueeze(0)

  output = model(token_ids, attention_mask = attention_mask)
  output_2 = model(token_ids_2, attention_mask = attention_mask_2)

  cosine_sim = cosine_similarity(output[1].detach().numpy(), output_2[1].detach().numpy())

  cosine_similarities.append(cosine_sim[0].item())

  #if df.iloc[i]['p(Hallucination)'] >= 0.66:
  if df.iloc[i]['p(Hallucination)'] == 1:
    hallucinations_cosines.append(cosine_sim[0].item())
    hallucinations.append(i)

  output_pairs.append((output, output_2))

  output_array = np.array(output_pairs)

In [ ]:
#output_array[i, 0] is the embedding for the i-th sentence (hyp).
#output_array[i, 1] is the embedding for the i-th sentence_2 (tgt).

In [ ]:
threshold = np.mean(hallucinations_cosines)

hallucinations_pred = []

for i in range(0,len(df)):
    similarity = cosine_similarities[i]
    if similarity < threshold:
        hallucinations_pred.append(i)

counter1 = Counter(hallucinations)
counter2 = Counter(hallucinations_pred)

intersection = counter1 & counter2

total_mutual_elements = sum(intersection.values())


print(f'Threshold: {threshold}')
print(f'Total Hallucinations: {len(hallucinations)}')
print(f'Total Hallucinations Predicted: {len(hallucinations_pred)}')
print('Percentage of correctly predicted hallucinations:' , (total_mutual_elements / len(hallucinations)) * 100 , '%')
print('Correct Hallucinations:', intersection.keys())
print('Number of correct Hallucinations:', len(intersection.keys()))